In [1]:
import numpy as np
from tqdm.notebook import tqdm

tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil

from joblib import Parallel, delayed

/home/kaor/.local/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# !git clone https://github.com/ultralytics/yolov5

In [3]:
df = pd.read_csv("~/whale/subset_1_df.csv")
df['image_path'] = "/home/kaor/whale/yolo/images/"+df['image']
df['image_r_path'] = "/home/kaor/whale/train_images/"+df['image']
train_files = list(df.image_path)
train_files[0:5]

['/home/kaor/whale/yolo/images/cf4f3fe6605328.jpg',
 '/home/kaor/whale/yolo/images/de84b48e1d0aaf.jpg',
 '/home/kaor/whale/yolo/images/dcf21bf0ffe7e9.jpg',
 '/home/kaor/whale/yolo/images/fc769209af8ca4.jpg',
 '/home/kaor/whale/yolo/images/4895e2b46aa757.jpg']

In [4]:
# !mkdir /home/kaor/whale/yolo/
# !mkdir /home/kaor/whale/yolo/images/
# !mkdir /home/kaor/whale/yolo/labels/

In [5]:
# !cp -r /home/kaor/whale/yolo_annotations/yolo_annotations/*.txt /home/kaor/whale/yolo/labels/

In [6]:
# !ls /home/kaor/whale/train_images/cf4f3fe6605328.jpg

In [7]:
# for i,row in df.iterrows():
#     shutil.copy(str(row['image_r_path']),str(row['image_path']))

In [8]:
# !ls /home/kaor/whale/yolo/images/

In [9]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=6, random_state=42, shuffle=True)
df = df.reset_index(drop=True)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
    df.loc[val_idx, 'fold'] = fold
df.fold.value_counts()

0    256
5    255
4    255
3    255
2    255
1    255
Name: fold, dtype: int64

In [10]:
FOLD = 1
train_files = []
val_files   = []
train_df = df.query("fold!=@FOLD")
valid_df = df.query("fold==@FOLD")
train_files += list(train_df.image_path.unique())
val_files += list(valid_df.image_path.unique())
len(train_files), len(val_files)

(1275, 256)

In [11]:
!ls /home/kaor/whale/yolo/labels/cf4f3fe6605328.txt

ls: cannot access /home/kaor/whale/yolo/labels/cf4f3fe6605328.txt: No such file or directory


In [12]:
import yaml

cwd = '/home/kaor/whale/yolo'

with open(os.path.join( cwd , 'train.txt'), 'w') as f:
    for path in train_df.image_path.tolist():
        if os.path.exists(path.replace("images","labels").replace("jpg","txt")):
            f.write(path+'\n')
        else:
            print(path)
            
with open(os.path.join(cwd , 'val.txt'), 'w') as f:
    for path in valid_df.image_path.tolist():
        if os.path.exists(path.replace("images","labels").replace("jpg","txt")):
            f.write(path+'\n')
        else:
            print(path)

data = dict(
    path  = cwd,
    train =  os.path.join( cwd , 'train.txt') ,
    val   =  os.path.join( cwd , 'val.txt' ),
    nc    = 1,
    names = ['whale'],
    )

with open(os.path.join( cwd , 'happywhale.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join( cwd , 'happywhale.yaml'), 'r')
print('\nyaml:')
print(f.read())

/home/kaor/whale/yolo/images/cf4f3fe6605328.jpg
/home/kaor/whale/yolo/images/9a8b39b1b6d472.jpg
/home/kaor/whale/yolo/images/9c96a72250084c.jpg
/home/kaor/whale/yolo/images/60de2ea6b649a8.jpg
/home/kaor/whale/yolo/images/abe8de4c14ddbc.jpg
/home/kaor/whale/yolo/images/a6d9a9d4a8a444.jpg
/home/kaor/whale/yolo/images/dda4bba985a0f0.jpg
/home/kaor/whale/yolo/images/dd46ff0597c1aa.jpg
/home/kaor/whale/yolo/images/aa16947ce5316e.jpg
/home/kaor/whale/yolo/images/a8e7f0347b7c62.jpg
/home/kaor/whale/yolo/images/baf82000789858.jpg
/home/kaor/whale/yolo/images/ebc7e212d6e505.jpg
/home/kaor/whale/yolo/images/8c5221ff535f10.jpg
/home/kaor/whale/yolo/images/b58f4f1d9f6247.jpg
/home/kaor/whale/yolo/images/f8ec1df54bba10.jpg
/home/kaor/whale/yolo/images/af64d05f30daca.jpg
/home/kaor/whale/yolo/images/fdf98dfb01d768.jpg
/home/kaor/whale/yolo/images/c45d288419aba6.jpg

yaml:
names:
- whale
nc: 1
path: /home/kaor/whale/yolo
train: /home/kaor/whale/yolo/train.txt
val: /home/kaor/whale/yolo/val.txt



In [13]:
import sys
sys.path.append("./yolov5")

In [14]:
DIM = 768
BATCH = 8
EPOCHS = 10
MODEL = "yolov5l"

In [15]:
!~/.conda/envs/openmmlab/bin/python yolov5/train.py --img {DIM}\
--batch {BATCH}\
--epochs {EPOCHS}\
--data /home/kaor/whale/yolo/happywhale.yaml\
--weights {MODEL}.pt --cache

train: weights=yolov5l.pt, cfg=, data=/home/kaor/whale/yolo/happywhale.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=8, imgsz=768, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
requirements: matplotlib>=3.2.2 not found and is required by YOLOv5, attempting auto-update...

requirements: pandas>=1.1.4 not found and is required by YOLOv5, attempting auto-update...

requirements: 2 packages updated per /sise/home/kaor/whale_code/yolo/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for

In [19]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5','custom',path = 'yolov5/runs/train/exp2/weights/best.pt',)  # or yolov5m, yolov5l, yolov5x, custom

Using cache found in /home/kaor/.cache/torch/hub/ultralytics_yolov5_master


requirements: matplotlib>=3.2.2 not found and is required by YOLOv5, attempting auto-update...

requirements: pandas>=1.1.4 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-12-20 torch 1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)




requirements: 2 packages updated per /sise/home/kaor/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 367 layers, 46108278 parameters, 0 gradients
Adding AutoShape... 


In [103]:
model.conf = 0.05

In [ ]:
%matplotlib inline
from tqdm import tqdm
r = glob.glob("/home/kaor/whale/train_images/*jpg")
df_list = []
# for z in range(30,40):
#     # plt.figure(figsize=[20,20])
#     # s = 1
if True:
    for i in tqdm(range(0,len(r))):
        # plt.subplot(3,3,s)
        # s+=1
        img= cv2.imread(r[i])[:,:,::-1]
        res = model(img)
        if res.xyxy[0].shape[0] != 0:
            d_f = res.pandas().xyxy[0]
            d_f['image'] = r[i].split("/")[-1]
            df_list.append(d_f)
pd.concat(df_list).to_csv("train_bbox.csv",index=False)
    #     img = cv2.imread(r[i])
    #     if res.xyxy[0].shape[0] != 0:
    #         for row in res.pandas().xyxy[0].values:
    #             x0,y0,x1,y1,c,cl,n = row
    #             img = cv2.rectangle(img,(int(x0),int(y0)),(int(x1),int(y1)),(255,0,0),int(max(img.shape[0],img.shape[1])/60))
    #             break
    #     plt.imshow(img[:,:,::-1])
    # plt.show()

 13%|█▎        | 6571/51033 [17:23<2:37:56,  4.69it/s]

In [ ]:
r = glob.glob("/home/kaor/whale/test_images/*jpg")
df_list = []
if True:
    for i in tqdm(range(0,len(r))):
        img= cv2.imread(r[i])[:,:,::-1]
        res = model(img)
        if res.xyxy[0].shape[0] != 0:
            d_f = res.pandas().xyxy[0]
            d_f['image'] = r[i].split("/")[-1]
            df_list.append(d_f)
pd.concat(df_list).to_csv("test_bbox.csv",index=False)